<a href="https://colab.research.google.com/github/sidagarwal-labs/Analyzing-Yelp-Reviews---Philadelphia-Restaurants/blob/main/1_Topic_phrase%2Bsentiment%2Bsubtheme_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Please be aware tha that I connected to my google drive for the files and used my own person API key for Open AI when running this code, it will probably fail without changes. In order to run you will need your own API Key

In [ ]:
import os
import json
import pandas as pd
pd.set_option('display.max_columns', None)

from google.colab import drive, ai, userdata
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df1 = pd.read_parquet("/content/drive/MyDrive/yelp_final_project/LLM/output/20_sampled_review_topics.parquet")
df2 = pd.read_parquet("/content/drive/MyDrive/yelp_final_project/LLM/output/2017_sampled_review_topics.parquet")
df3 = pd.read_parquet("/content/drive/MyDrive/yelp_final_project/LLM/output/2018_sampled_review_topics.parquet")
df4 = pd.read_parquet("/content/drive/MyDrive/yelp_final_project/LLM/output/2021_2022_sampled_review_topics.parquet")

processed_df = pd.concat([df1, df2, df3, df4])
processed_df['review_id'] = processed_df['review_id'].astype(str)

In [ ]:
review_df = pd.read_parquet("/content/drive/MyDrive/yelp_final_project/yelp_philadelphia/review.parquet")
review_df['review_id'] = review_df['review_id'].astype(str)

In [ ]:
all_df = processed_df.merge(review_df, on='review_id', how='left', suffixes=('', '_review')).drop(columns=['stars','useful','funny','cool','text'])

In [ ]:
all_df['year'] = pd.to_datetime(all_df['date']).dt.year
distinct_reviews_per_year = all_df.groupby('year')['review_id'].nunique()
print(distinct_reviews_per_year)

year
2017    845
2018    920
2019     12
2020     13
2021    512
2022    510
Name: review_id, dtype: int64


In [ ]:
# reviews_df = pd.read_parquet("/content/drive/MyDrive/yelp_final_project/yelp_philadelphia/review.parquet")
# reviews_not_already_processed_df
df = review_df[~review_df['review_id'].isin(all_df['review_id'])]

# Filtering to the dates we are looking at
df = df[(df['date'] >= '2019-01-01') & (df['date'] < '2021-01-01')]

# Adding a date column for stratified sampling
df["year"] = pd.to_datetime(df["date"]).dt.year

# Generate a sample stratified by year
sampled_df = (df.groupby("year").apply(lambda g: g.sample(min(500, len(g)), random_state=42)).reset_index(drop=True))

# Randomly order the df so all years get run in case quota runs out
sampled_df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

/tmp/ipython-input-3118628983.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = (df.groupby("year").apply(lambda g: g.sample(min(500, len(g)), random_state=42)).reset_index(drop=True))


In [ ]:
sampled_df.groupby('year')['review_id'].nunique()

,review_id
year,
2019,500
2020,500


In [ ]:
system_instruction = """
You are a restaurant review analyst. Your job is to extract topics from restaurant reviews
and assign a sentiment and category to each statement. A review may contain multiple topics.

Rules:
- Identify EVERY distinct topic mentioned. Do not merge unrelated aspects.
- Return ALL topics (food, service, price, ambience, cleanliness, staff, wait time, location, etc.).
- For each topic include:
    - "phrase": short quote or paraphrase from the review
    - "sentiment": positive, negative, or neutral connotation of phrase
    - "category": concise label (e.g., "service", "food quality", "price")

Output Format (MANDATORY):
Return ONLY valid JSON in this exact shape:

{
  "topics": [
    {
      "phrase": "...",
      "sentiment": "...",
      "category": "..."
    }
  ]
}

Do NOT include any text outside the JSON. No commentary. No markdown. DO NOT include "```json"
in your output!
"""


In [ ]:
from pydantic import BaseModel
from typing import List
from openai import OpenAI

client = OpenAI(api_key=userdata.get("OPEN_API_KEY"))

class Topic(BaseModel):
    phrase: str
    sentiment: str
    category: str

class ReviewTopics(BaseModel):
    topics: List[Topic]


def extract_topics(review_text: str):
  # response_str = ai.generate_text(f"{system_instruction}\n\n{review_text}")

  response = client.chat.completions.create(
      model="gpt-4o-mini",
      temperature=0,
      messages=[
          {"role": "system", "content": system_instruction},
          {"role": "user", "content": review_text}
      ])
  response_str = response.choices[0].message.content

  # Parse the JSON string and validate with Pydantic
  review_topics = ReviewTopics.model_validate_json(response_str)

  return review_topics.model_dump() # Return as a dictionary


def process_reviews_df(df, text_col="text", id_col="review_id"):
    rows = []
    row_num = 0
    for row in df.itertuples(index=False):
        review_id = getattr(row, id_col)
        text = getattr(row, text_col)
        topics_json = {"topics": []} # Initialize topics_json with a default empty structure
        try:
          topics_json = extract_topics(text)
        except Exception as e:
          print(f"Error processing review_id {review_id}: {e}")

        row_num += 1
        print(row_num, topics_json)

        rows.append({
            "review_id": review_id,
            "raw_text": text,
            "topics": topics_json["topics"]
        })
    return rows


def flatten_topics(review_outputs):
    flat = []
    for r in review_outputs:
        for t in r["topics"]:
            flat.append({
                "review_id": r["review_id"],
                "review_text": r["raw_text"],
                "topic_phrase": t["phrase"],
                "topic_category": t["category"],
                "topic_sentiment": t["sentiment"]

            })
    return pd.DataFrame(flat)

In [ ]:
review_outputs = process_reviews_df(sampled_df, text_col="text", id_col="review_id")

1 {'topics': [{'phrase': 'Nice people', 'sentiment': 'positive', 'category': 'staff'}, {'phrase': 'good food', 'sentiment': 'positive', 'category': 'food quality'}, {'phrase': 'Pizza is consistently delicious', 'sentiment': 'positive', 'category': 'food quality'}, {'phrase': 'High quality ingredients', 'sentiment': 'positive', 'category': 'food quality'}, {'phrase': "it's the best", 'sentiment': 'positive', 'category': 'food quality'}]}
2 {'topics': [{'phrase': 'Great service even in these trying times.', 'sentiment': 'positive', 'category': 'service'}, {'phrase': 'excellent food and atmosphere.', 'sentiment': 'positive', 'category': 'food quality'}, {'phrase': 'I highly recommend the mac and cheese.', 'sentiment': 'positive', 'category': 'food quality'}, {'phrase': 'some creamy but god bless the chef', 'sentiment': 'positive', 'category': 'food quality'}]}
3 {'topics': [{'phrase': 'most deliciously ratchet, wild, outrageously fun tiny dance floor', 'sentiment': 'positive', 'category':

In [ ]:
topics_df = flatten_topics(review_outputs)

topics_df.head()

,review_id,review_text,topic_phrase,topic_category,topic_sentiment
0,r_3vL1iRQxNGSUMQ99rAjg,Nice people making good food. Pizza is consist...,Nice people,staff,positive
1,r_3vL1iRQxNGSUMQ99rAjg,Nice people making good food. Pizza is consist...,good food,food quality,positive
2,r_3vL1iRQxNGSUMQ99rAjg,Nice people making good food. Pizza is consist...,Pizza is consistently delicious,food quality,positive
3,r_3vL1iRQxNGSUMQ99rAjg,Nice people making good food. Pizza is consist...,High quality ingredients,food quality,positive
4,r_3vL1iRQxNGSUMQ99rAjg,Nice people making good food. Pizza is consist...,it's the best,food quality,positive


In [ ]:
topics_df.to_parquet("/content/drive/MyDrive/yelp_final_project/2019_2020_sampled_review_topics.parquet", index=False)

In [ ]:
topics_df.shape

(6661, 5)